In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

In [18]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 128
skip_window = 5

In [15]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper(embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [34]:
seq_embeddings = np.load('results/char_embedding.npy')
sem_embeddings = np.load('results/word2vec_embedding.npy')
seq_em = normalize(seq_embeddings)
sem_em = normalize(sem_embeddings)

D:\Library\Projects\amharic_word_embedding\data_handle.py:217: RuntimeWarning: invalid value encountered in true_divide
  return array / norms


In [106]:
dot_prods = np.einsum('ij,ij->i', seq_embeddings, sem_embeddings).reshape((-1, 1))
seq_norms = np.linalg.norm(seq_embeddings, keepdims=True)**2
sem_norms = np.linalg.norm(sem_embeddings, keepdims=True)**2
for_seq = dot_prods/seq_norms
for_sem = dot_prods/sem_norms
seq_proj = for_seq * seq_embeddings
sem_proj = for_sem * sem_embeddings
print(dot_prods.mean(), (seq_norms**.5).mean(), (sem_norms **.5).mean())

-23.089790721731596 39650.2555692447 1413.9790310317721


In [119]:
seq = normalize(0.0357*seq_embeddings + sem_embeddings)
sem = normalize(sem_embeddings )

# seq = 10*seq_em + sem_em
# seq = normalize(seq_embeddings)
# sem = normalize(sem_embeddings)


D:\Library\Projects\amharic_word_embedding\data_handle.py:217: RuntimeWarning: invalid value encountered in true_divide
  return array / norms


In [139]:
for i in np.arange(.75, 0.55, -0.01):
    seq = np.concatenate([i*normalize(seq_embeddings), normalize(sem_embeddings)], axis=1)
    print(evaluate(seq, word2int, embed_size*2))

D:\Library\Projects\amharic_word_embedding\data_handle.py:217: RuntimeWarning: invalid value encountered in true_divide
  return array / norms
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 72.48% semantic: 6.49% syntactic: 8.33% 
None
anomaly: 72.48% semantic: 6.49% syntactic: 8.33% 
None
anomaly: 72.48% semantic: 6.49% syntactic: 8.33% 
None
anomaly: 72.48% semantic: 6.49% syntactic: 7.58% 
None
anomaly: 72.48% semantic: 6.49% syntactic: 7.58% 
None
anomaly: 72.48% semantic: 7.03% syntactic: 7.58% 
None
anomaly: 71.56% semantic: 7.03% syntactic: 7.58% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 7.58% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 7.58% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 6.82% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 6.82% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 6.82% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 6.06% 
None
anomaly: 71.56% semantic: 7.57% syntactic: 6.06% 
None
anomaly: 71.56% semantic: 8.11% syntactic: 6.06% 
None
anomaly: 71.56% semantic: 8.11% syntactic: 6.06% 
None
anomaly: 71.56% semantic: 8.11% syntactic: 6.06% 
None
anomaly: 71.56% semantic: 8.11% syntactic: 6.06% 
None
anomaly: 7

In [110]:
evaluate(sem, word2int, embed_size)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 69.72% semantic: 11.35% syntactic: 1.52% 
